In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import explained_variance_score, \
    mean_absolute_error, \
    median_absolute_error
import tensorflow as tf
import joblib
import matplotlib.pyplot as plt      
import numpy as np        
import pandas
import matplotlib as plt
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import csv
from datetime import date, timedelta
from collections import defaultdict



In [3]:
Path="Raw_Data/grouped_dates_monterey_avg.csv"
df=pd.read_csv(Path)
monterey_df=pd.DataFrame(df)


In [4]:
monterey_df.head()

,DATE,LATITUDE,LONGITUDE,ELEVATION,EMNT,EMXT,TAVG
0,2000-03,36.574080,-121.786360,64.260,4.575,26.125000,11.800000
1,2000-04,36.574080,-121.786360,64.260,5.025,28.475000,13.750000
2,2000-05,36.574080,-121.786360,64.260,5.125,29.450000,14.625000
3,2000-06,36.569425,-121.769075,71.025,7.400,29.433333,15.866667
4,2000-07,36.574080,-121.786360,64.260,8.475,21.825000,15.400000


In [5]:
monterey_df[['Year', 'Month']]=monterey_df['DATE'].str.split('-',1,expand=True)

In [6]:
monterey_yr_mo_avg=monterey_df[['Year','Month','TAVG']]

In [7]:
monterey_yr_mo_avg_pivot=monterey_yr_mo_avg.pivot(index='Year', columns='Month', values=['TAVG'])

In [8]:
monterey_yr_mo_avg_pivot.head()

TAVG                                                         \
Month         01         02         03         04         05         06   
Year                                                                      
2000         NaN        NaN  11.800000  13.750000  14.625000  15.866667   
2001   10.025000  10.675000  12.575000  11.050000  15.450000  15.700000   
2002    8.866667  11.685714  11.433333  12.560000  13.066667  14.542857   
2003   13.571429  11.228571  12.485714  11.916667  14.050000  15.960000   
2004   10.033333  10.785714  14.542857  13.771429  14.428571  15.085714   

                                                                         
Month         07         08         09         10         11         12  
Year                                                                     
2000   15.400000  16.025000  17.400000  14.725000  10.833333  11.900000  
2001   15.925000  15.925000  15.975000  15.700000  13.216667  10.666667  
2002   16.085714  16.428571  17.000000  14.871429  14.514286  11.414286  
2003   16.800000  17.828571  18.342857  16.557143  12.000000  11.428571  
2004   16.700000  17.760000  17.880000  15.140000  12.285714  11.228571

In [9]:
monterey_yr_mo_avg_pivot.columns=monterey_yr_mo_avg_pivot.columns.droplevel(0)

In [10]:
monterey_butterfly_count=(20000,31203,11593,68979,54481,37540,59957,15426,31063,4735,8634,27788,29048,35772,55879,27945,64974,33657,1890,2792,58)

monterey_yr_mo_avg_pivot['Count']=monterey_butterfly_count

In [11]:
monterey_yr_mo_avg_pivot.head()

Month,01,02,03,04,05,06,07,08,09,10,11,12,Count
Year,,,,,,,,,,,,,
2000,NaN,NaN,11.800000,13.750000,14.625000,15.866667,15.400000,16.025000,17.400000,14.725000,10.833333,11.900000,20000
2001,10.025000,10.675000,12.575000,11.050000,15.450000,15.700000,15.925000,15.925000,15.975000,15.700000,13.216667,10.666667,31203
2002,8.866667,11.685714,11.433333,12.560000,13.066667,14.542857,16.085714,16.428571,17.000000,14.871429,14.514286,11.414286,11593
2003,13.571429,11.228571,12.485714,11.916667,14.050000,15.960000,16.800000,17.828571,18.342857,16.557143,12.000000,11.428571,68979
2004,10.033333,10.785714,14.542857,13.771429,14.428571,15.085714,16.700000,17.760000,17.880000,15.140000,12.285714,11.228571,54481


In [56]:
monterey_on_season=monterey_yr_mo_avg_pivot[['11','12','01','02','03','Count']]

In [13]:

monterey_off_season=monterey_yr_mo_avg_pivot[['04','05','06','07','08']]

In [57]:
monterey_on_season

Month,11,12,01,02,03,Count
Year,,,,,,
2000,10.833333,11.900000,NaN,NaN,11.800000,20000
2001,13.216667,10.666667,10.025000,10.675000,12.575000,31203
2002,14.514286,11.414286,8.866667,11.685714,11.433333,11593
2003,12.000000,11.428571,13.571429,11.228571,12.485714,68979
2004,12.285714,11.228571,10.033333,10.785714,14.542857,54481
2005,14.600000,12.000000,10.900000,12.485714,13.585714,37540
2006,12.928571,10.871429,11.014286,12.014286,9.914286,59957
2007,13.250000,9.416667,9.028571,10.971429,12.700000,15426
2008,14.333333,9.716667,9.600000,10.633333,11.300000,31063


In [58]:
monterey_on_season['Y/N']=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]


Month,11,12,01,02,03,Count,Y/N
Year,,,,,,,
2000,10.833333,11.900000,NaN,NaN,11.800000,20000,1
2001,13.216667,10.666667,10.025000,10.675000,12.575000,31203,1
2002,14.514286,11.414286,8.866667,11.685714,11.433333,11593,1
2003,12.000000,11.428571,13.571429,11.228571,12.485714,68979,1
2004,12.285714,11.228571,10.033333,10.785714,14.542857,54481,1
2005,14.600000,12.000000,10.900000,12.485714,13.585714,37540,1
2006,12.928571,10.871429,11.014286,12.014286,9.914286,59957,1
2007,13.250000,9.416667,9.028571,10.971429,12.700000,15426,1
2008,14.333333,9.716667,9.600000,10.633333,11.300000,31063,1


In [15]:
monterey_off_season['Count']=(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)

In [16]:
monterey_off_season['Y/N']=(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
monterey_off_season.head()


Month,04,05,06,07,08,N,Count,Y/N
Year,,,,,,,,
2000,13.750000,14.625000,15.866667,15.400000,16.025000,0,0,0
2001,11.050000,15.450000,15.700000,15.925000,15.925000,0,0,0
2002,12.560000,13.066667,14.542857,16.085714,16.428571,0,0,0
2003,11.916667,14.050000,15.960000,16.800000,17.828571,0,0,0
2004,13.771429,14.428571,15.085714,16.700000,17.760000,0,0,0


In [60]:
monterey_on_season.head()

Month,11,12,01,02,03,Count,Y/N
Year,,,,,,,
2000,10.833333,11.900000,NaN,NaN,11.800000,20000,1
2001,13.216667,10.666667,10.025000,10.675000,12.575000,31203,1
2002,14.514286,11.414286,8.866667,11.685714,11.433333,11593,1
2003,12.000000,11.428571,13.571429,11.228571,12.485714,68979,1
2004,12.285714,11.228571,10.033333,10.785714,14.542857,54481,1


In [61]:
monterey_on_season=monterey_on_season.rename(columns={'11':'month_1','12':'month_2','01':"month_3",'02':'month_4','03':'month_5'})
montere_off_season=monterey_off_season.rename(columns={'04':'month_1','05':'month_2','06':'month_3','07':'month_4','08':'month_5'})

In [90]:
good_data_monterey=pd.concat([montere_off_season,monterey_on_season])

In [91]:
good_data_monterey

,month_1,month_2,month_3,month_4,month_5,N,Count,Y/N
Year,,,,,,,,
2000,13.750000,14.625000,15.866667,15.400000,16.025000,0.0,0,0
2001,11.050000,15.450000,15.700000,15.925000,15.925000,0.0,0,0
2002,12.560000,13.066667,14.542857,16.085714,16.428571,0.0,0,0
2003,11.916667,14.050000,15.960000,16.800000,17.828571,0.0,0,0
2004,13.771429,14.428571,15.085714,16.700000,17.760000,0.0,0,0
2005,12.385714,14.842857,14.885714,16.342857,16.228571,0.0,0,0
2006,12.457143,14.242857,16.228571,17.128571,15.900000,0.0,0,0
2007,12.050000,13.283333,14.583333,16.300000,16.783333,0.0,0,0
2008,12.116667,13.350000,15.216667,16.016667,16.266667,0.0,0,0


In [92]:
good_data_monterey=good_data_monterey.dropna()

In [88]:
#good_data_monterey=good_data_monterey.reset_index()

In [94]:
good_data_monterey[]

Year
2000    13.750000
2001    11.050000
2002    12.560000
2003    11.916667
2004    13.771429
2005    12.385714
2006    12.457143
2007    12.050000
2008    12.116667
2009    12.516667
2010    11.766667
2011    12.066667
2012    12.700000
2013    13.220000
2014    13.980000
2015    13.020000
2016    14.580000
2017    13.825000
2018    13.240000
2019    14.266667
Name: month_1, dtype: float64

In [37]:
from sklearn.utils import shuffle
monterey_shuffle=shuffle(good_data_monterey)

In [38]:
X=monterey_shuffle[['month_1','month_2','month_3','month_4','month_5','Count','Y/N']]
y=monterey_shuffle['Count']


In [40]:
X.values,y.values

(array([[12.05      , 13.28333333, 14.58333333, 16.3       , 16.78333333,
          0.        ,  0.        ],
        [13.22      , 14.72      , 16.15      , 16.38      , 16.92      ,
          0.        ,  0.        ],
        [13.825     , 14.525     , 16.025     , 16.73333333, 17.6       ,
          0.        ,  0.        ],
        [12.11666667, 13.35      , 15.21666667, 16.01666667, 16.26666667,
          0.        ,  0.        ],
        [13.98      , 15.7       , 15.68      , 17.82      , 17.94      ,
          0.        ,  0.        ],
        [12.06666667, 12.61666667, 14.4       , 16.02      , 15.46      ,
          0.        ,  0.        ],
        [12.38571429, 14.84285714, 14.88571429, 16.34285714, 16.22857143,
          0.        ,  0.        ],
        [13.24      , 14.18      , 15.45      , 17.6       , 16.925     ,
          0.        ,  0.        ],
        [11.76666667, 12.48333333, 14.85      , 15.26666667, 15.43333333,
          0.        ,  0.        ],
        [1

In [41]:
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [44]:
classifier = LogisticRegression()
classifier

LogisticRegression()

In [45]:
classifier.fit(X_train, y_train)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

In [1]:
 predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

NameError: name 'classifier' is not defined

In [10]:
Sacramento=pd.read_csv("Raw_Data/grouped_dates_sacramento_avg.csv")
sacramento_df=pd.DataFrame(Sacramento)
Sandiego=pd.read_csv("Raw_Data/grouped_dates_sandiego_avg.csv")
sandiego_df=Sandiego
Santamaria=pd.read_csv("Raw_Data/grouped_dates_santamaria_avg.csv")
santamaria_df=Santamaria
Santa_rosa=pd.read_csv("Raw_Data/grouped_dates_santarosa_avg.csv")
santarosa_df=Santa_rosa
Stockton=pd.read_csv("Raw_Data/grouped_dates_stockton_avg.csv")
stockton_df=Stockton

In [11]:
sacramento_df[['Year', 'Month']]=sacramento_df['DATE'].str.split('-',1,expand=True)
sacramento_yr_mo_avg=sacramento_df[['Year','Month','TAVG']]
sacramento_yr_mo_avg_pivot=sacramento_yr_mo_avg.pivot(index='Year', columns='Month', values=['TAVG'])

In [12]:
sacramento_yr_mo_avg_pivot.head()

TAVG                                                         \
Month         01         02         03         04         05         06   
Year                                                                      
2000    9.466667  11.000000  13.266667  16.433333  19.433333  23.633333   
2001    7.966667   9.566667  14.433333  13.933333  22.800000  23.500000   
2002    7.333333  10.633333  11.733333  15.100000  18.766667  23.166667   
2003   10.466667  10.233333  13.500000  12.600000  18.766667  23.033333   
2004    8.366667  10.533333  15.800000  17.166667  19.800000  22.966667   

                                                                        
Month         07         08         09         10         11        12  
Year                                                                    
2000   23.000000  23.933333  22.033333  16.666667   9.333333  8.633333  
2001   23.633333  24.200000  22.200000  19.066667  13.466667  9.400000  
2002   25.066667  23.600000  22.966667  17.200000  12.600000  9.900000  
2003   26.766667  24.333333  23.500000  19.400000  10.766667  9.600000  
2004   24.566667  24.666667  22.500000  17.000000  10.933333  8.466667

In [13]:
sacramento_butterfly_count = (0,0,0,0,0,0,0,0,0,0,0,0,0,210,16,5,17,0,10)

In [14]:
sandiego_df[['Year', 'Month']]=sandiego_df['DATE'].str.split('-',1,expand=True)
sandiego_yr_mo_avg=sandiego_df[['Year','Month','TAVG']]
sandiego_yr_mo_avg_pivot=sandiego_yr_mo_avg.pivot(index='Year', columns='Month', values=['TAVG'])

In [15]:
sandiego_yr_mo_avg_pivot.head()

TAVG                                                         \
Month         01         02         03         04         05         06   
Year                                                                      
2000   13.658824  13.406667  13.368750  16.481250  18.740000  21.021429   
2001   11.022222  11.255556  14.042105  13.752632  18.505263  20.794444   
2002   11.868182  13.495000  12.829412  14.795000  16.563158  19.244444   
2003   15.664706  13.336842  14.194737  13.673684  16.688889  18.733333   
2004   12.252632  11.678947  16.300000  16.364706  19.029412  19.287500   

                                                                         
Month         07         08         09         10         11         12  
Year                                                                     
2000   21.852941  23.529412  21.922222  17.300000  13.029412  13.964706  
2001   21.476190  22.566667  21.500000  19.131579  15.221053  11.342105  
2002   21.463158  21.566667  22.066667  17.100000  16.605263  11.778947  
2003   23.178947  24.315000  22.300000  20.486667  14.440000  12.300000  
2004   22.680000  22.378947  22.406250  17.443750  13.372222  12.817647

In [16]:
sandiego_butterfly_count = (348,78,40,50,260,902,395,51,723,73,372,92,292,144,232,194,50,60,12,22,30)

In [17]:
santamaria_df[['Year', 'Month']]=santamaria_df['DATE'].str.split('-',1,expand=True)
santamaria_yr_mo_avg=santamaria_df[['Year','Month','TAVG']]
santamaria_yr_mo_avg_pivot=santamaria_yr_mo_avg.pivot(index='Year', columns='Month', values=['TAVG'])

In [18]:
santamaria_yr_mo_avg_pivot.tail()

TAVG                                                         \
Month         01         02         03         04         05         06   
Year                                                                      
2016   12.500000  16.250000  14.600000  16.200000  14.400000  16.900000   
2017   10.900000  11.966667  14.400000  14.800000  15.100000  16.566667   
2018   14.733333  12.650000  12.666667  14.133333  14.366667  16.300000   
2019   12.633333   9.533333  13.100000  14.966667  14.566667  17.266667   
2020   12.500000  13.300000  11.900000        NaN        NaN        NaN   

                                                                     
Month     07         08         09         10         11         12  
Year                                                                 
2016   16.95  17.166667  18.800000  17.500000  15.066667  11.333333  
2017   17.95  18.400000  20.100000  18.900000  16.500000  14.500000  
2018   19.10  19.300000  17.266667  18.333333  16.166667  12.500000  
2019   17.70  18.633333  20.100000  18.433333  14.833333  12.300000  
2020     NaN        NaN        NaN        NaN        NaN        NaN

In [19]:
santarosa_df[['Year', 'Month']]=santarosa_df['DATE'].str.split('-',1,expand=True)
santarosa_yr_mo_avg=santarosa_df[['Year','Month','TAVG']]
santarosa_yr_mo_avg_pivot=santarosa_yr_mo_avg.pivot(index='Year', columns='Month', values=['TAVG'])

In [20]:
santarosa_yr_mo_avg_pivot.head()

TAVG                                                         \
Month         01         02         03         04         05         06   
Year                                                                      
2000    9.671429  10.875000  12.285714  14.542857  17.033333  18.442857   
2001    8.037500   9.333333  13.257143  12.200000  19.350000  20.375000   
2002    8.162500  10.966667  11.285714  13.750000  15.362500  20.100000   
2003   11.344444  10.425000  12.800000  11.177778  16.188889  19.414286   
2004    9.100000   9.962500  15.555556  15.087500  16.950000  19.342857   

                                                                        
Month         07         08         09         10         11        12  
Year                                                                    
2000   18.925000  19.700000  19.757143  15.475000  10.228571  9.575000  
2001   18.900000  20.344444  18.675000  17.875000  12.828571  9.487500  
2002   20.342857  20.285714  19.750000  16.188889  13.288889  9.812500  
2003   21.214286  20.800000  20.500000  18.314286  10.411111  9.162500  
2004   20.287500  21.271429  20.942857  15.633333  11.587500  9.528571

In [21]:
santa_rosa_butterfly_count= (0,0,0,0,0,0,0,0,0,0,600,20,5,8,911,50,18,5,0,7)

In [22]:
stockton_df[['Year', 'Month']]=stockton_df['DATE'].str.split('-',1,expand=True)
stockton_yr_mo_avg=stockton_df[['Year','Month','TAVG']]
stockton_yr_mo_avg_pivot=stockton_yr_mo_avg.pivot(index='Year', columns='Month', values=['TAVG'])

In [23]:
stockton_yr_mo_avg_pivot.head()

TAVG                                                     \
Month         01         02         03     04         05         06   
Year                                                                  
2000    9.866667  11.600000  13.166667  16.10  18.933333  23.166667   
2001    7.200000   9.700000  13.966667  13.60  22.100000  23.433333   
2002    7.033333  10.633333  11.950000  15.15  18.600000  23.000000   
2003   10.100000  10.200000  13.433333  13.40  18.950000  22.733333   
2004    8.133333  10.233333  15.800000  17.00  19.766667  22.766667   

                                                                    
Month         07     08         09         10         11        12  
Year                                                                
2000   22.633333  23.95  22.066667  16.866667   9.566667  8.266667  
2001   23.650000  24.20  22.133333  19.000000  12.966667  8.966667  
2002   24.600000  23.10  23.033333  17.533333  12.233333  9.300000  
2003   25.966667  23.90  23.300000  19.966667  11.100000  9.500000  
2004   24.233333  24.20  22.533333  16.700000  10.633333  8.233333